## Check GPU is on

In [1]:
# %tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

## Import standard library

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import model_from_json
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.resnet_v2 import preprocess_input
# from keras.applications.resnet_v2 import decode_predictions
from keras.applications import ResNet50V2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Generate Train Data

In [3]:
data_gen = preprocessing.image.ImageDataGenerator( 
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    fill_mode="nearest",
#     cval=0.0,
    horizontal_flip=True,
    validation_split=0.25,
    preprocessing_function=preprocess_input
)

In [4]:
train_datagen =  data_gen.flow_from_directory(
        '../input/shopee-product-detection-student/train/train/train',
        class_mode="categorical",
        color_mode="rgb",
        batch_size=128,
        target_size=(256, 256),
        shuffle=True,
        subset="training",
        interpolation="bilinear")

Found 79058 images belonging to 42 classes.


## Load trained model

In [5]:
# Load Latest Model
# load json and create model
json_file = open('../input/shopeeproductmodel/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../input/shopeeproductmodel/weights.04-1.53.ckpt")
print("Loaded model from disk")
model=loaded_model
model.summary()

Loaded model from disk
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 8, 8, 2048)        23564800  
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              134218752 
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 42)                43050     
Total params: 157,826,602
Trainable params: 157,781,162
Non-trainable params: 45,440
_________________________________________________________________


## Generate test data

Convert test.csv to data frame

In [6]:
testdf=pd.read_csv("../input/shopee-product-detection-student/test.csv",dtype=str)
testdf.head()

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,43
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,43
2,127f3e6d6e3491b2459812353f33a913.jpg,43
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,43
4,46d681a542f2c71be017eef6aae23313.jpg,43


Load image data

In [7]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return preprocess_input(input_img)

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return preprocess_input(input_img)

    return eraser

In [8]:
testgen = preprocessing.image.ImageDataGenerator( 
    rescale=1./255,
    preprocessing_function=get_random_eraser(v_l=0, v_h=255)
)
# get_random_eraser(v_l=0, v_h=255)

In [9]:
test_datagen = testgen.flow_from_dataframe(
    dataframe=testdf,
    directory="../input/shopee-product-detection-student/test/test/test",
    x_col="filename",
    y_col=None,
    batch_size=128,
    shuffle=False,
    class_mode=None,
    target_size=(256,256)
)

Found 12186 validated image filenames.


## Use predict generator to generator our predictions

In [10]:
prediction = model.predict_generator(
    test_datagen,
    steps=96,
)
# np.ceil(test_datagen.samples // 128)

## Save predictions to CSV file

In [11]:
predicted_class_indices=np.argmax(prediction,axis=1)
len(predicted_class_indices)

12186

In [12]:
# labels = (train_datagen.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [int(labels[k]) for k in predicted_class_indices]
# labels
labels = predicted_class_indices

In [13]:
filenames=test_datagen.filenames
len(filenames)
results=pd.DataFrame({"filename":filenames,
                      "category":labels})
results.to_csv("results.csv",index=False)